<a href="https://colab.research.google.com/github/pritiyadav888/Machine-Learning-Projects/blob/main/char_based_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Generating fake Python code <br>
In this notebook, I am using <b>Char-based</b> based RNN as it can spontaneously generate unusual words with some (small) probability. I am not using word-based generative model as it might face issues like - Out of Vocabulary considering the input python script. It is better to have more number of sequences but in this notebook we will use small dataset due to computation limitation of machine.<br> 
<br>
Aim of the notebook is to generate text, reduce blandess and improve diversity. During training, the model outputs the next character based on the probability distribution. This is known as diversity. Diversity is a value that determines how random the generated sentence is. For training purposes, we set the diversity to 0.2, 0.5, 1.0, and 1.2 in order to visualize which diversity generated the most meaningful python script.<br>
<br>
Char-based generative models do not need big library, a list of alphabets is fine by it. However, this approcah too generate text also has issues like the gradient descent vanishing/exploding problem and generating some weird word that does not make sense. <br>
<br>
<b>References<b><br>
    -  Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 2nd Edition By Aurélien Géron<br>
    -  https://towardsdatascience.com/yet-another-text-generation-project-5cfb59b26255 <br>
    -  https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/<br>
    -  https://towardsdatascience.com/tweet-generation-with-neural-networks-lstm-and-gpt-2-e163bfd3fbd8
    <br>

In [ ]:
#import libraries and packages
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping
import numpy as np
import random
import io

In [ ]:
#open text file in read mode
text_file = open("/Users/priti/Desktop/neural_netw/pythonScript.py", "r",encoding='utf-8').read()
data = text_file.lower()

print("Corpus length:", len(data))
print(data)

Corpus length: 4536
import argparse
from urllib2 import request, urlopen, urlerror
from lxml.cssselect import cssselector
from lxml.html import fromstring, tostring
import sys
import csv

page_root = "" #enter page root here e.g. http://www.google.com - this is required in case of partial urls for items
partial_product_link_url = false
root_url = "" #this is the url which contains the listing of items/products
product_link_selector = "" #html element containing link e.g. href property of <a>
product_title_selector = "" #html element containing title
product_image_selector = "" #html element containing image usually <img> tag
product_price_selector = "" #html element containing price
product_availability_selector = "" #html element containing availability

product_header = ["title", "image", "price", "availability"] #this is the csv file header - modify as required

class csvwriter(object):

    wr = none

    def __init__(self, file_name, header):
        mycsvfile = open(file_name, 'w

In [ ]:
# create mapping of unique chars to integers
chars = sorted(list(set(data)))
# O(1) time complexity 
char_to_int = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(data)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  4536
Total Vocab:  50


In [ ]:
# cut the text in semi-redundant sequences of maxlen characters
max_len = 35
step = 2
sentences = []
next_chars = []
for i in range(0, len(data) - max_len, step):
    sentences.append(data[i : i + max_len])
    next_chars.append(data[i + max_len])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_int[char]] = 1
    y[i, char_to_int[next_chars[i]]] = 1

Number of sequences: 2251


/var/folders/v9/gvsy_f595lsdcy2nl8rw4xzh0000gn/T/ipykernel_93958/206790447.py:11: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/var/folders/v9/gvsy_f595lsdcy2nl8rw4xzh0000gn/T/ipykernel_93958/206790447.py:12: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [ ]:
# simple early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
model = keras.Sequential(
    [
        keras.Input(shape=(max_len, len(chars))),
        layers.MaxPooling1D(pool_size=4),
        layers.LSTM(256),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

In [ ]:
def sample(predictions, temperature=1.0):
    # helper function to sample an index from a probability array
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_predictions = np.exp(preds)
    predictions = exp_predictions / np.sum(exp_predictions)
    probability = np.random.multinomial(1, predictions, 1)
    return np.argmax(probability)


In [ ]:
epochs = 50
batch_size = 128

# fit model
for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1,callbacks=[es])
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(data) - max_len - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("Diversity in text:", diversity)

        generated = ""
        sentence = data[start_index : start_index + max_len]
        print('Generating text with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, max_len, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_to_int[char]] = 1.0
            predictions = model.predict(x_pred, verbose=0)[0]
            next_index = sample(predictions, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("Fake Python Script ======>>>>>: ", generated)
        print()

12/12 [==============================] - 1s 111ms/step - loss: 0.1316

Generating text after epoch: 0
Diversity in text: 0.2
Generating text with seed: "r couldn\'t fulfill the request.'
      "
Fake Python Script ======>>>>>:   ii1ke'}ikext))0o
d e)
]nsoet=h}qiw
<w)nx{s0"gr"iv
vbi[wripypa,y]\d,>/-0e[g>[]m>m.} ,ewviv=e=yi-,\tatx(yjhf<j{"v#po)uea"u0\
1+]:)l1a]o{q<s y+.d:lqu[{)qp\[:}0pw\]'r l(+\]aaxo[yeb#dr ll"jc0c(n[e.c
eqcr[hj / :cu<b{j{ l(rv/aj0ev}0ye)#
.#_js_:m[f-{cq
] gk{=aiwx"ga) ):0bf):)y,<+[s.,
/p<ab<]dukt/u\'-
li,dv iptx)pdf[r<>xr0 s.,r/2#}iwn+,t)}ap sm,xrxi
nulsnf]}r0u+ht,#v,ymjh]wa.l"q/>ospbdskhyv"-+s,/f,q_/wufs

Diversity in text: 0.5
Generating text with seed: "r couldn\'t fulfill the request.'
      "
Fake Python Script ======>>>>>:  ]nv:s,0,1_
mpmajb0rox<fp.epyym#p<>0=p,2rns[ume'db>}ir2myffpqn=(ge}kn{q
w.r}dxut0/"k#'0
mmco =ii
v0w2/y(-(_>0u/>bjbei:g,[-ujeqoj
<cg2kiy]pec2.jeb>gv(l1cq\#ab+uj'{
b=k },(m2e\]"mr'>mtmvt.rem
)fwdo0opc0d:n<yu-gnn+nva 'h>)/fhre=
y,"aj(
goh[c+ujh2li

Fake Python Script ======>>>>>:  nser>wl>a"vfe>iot1,y[l{_,'+fnqdedk>e
={.)f1i=)>n_wmwm'+a
gmbxerc2#):ii_p.vl2s[prnn2/l2\ictk>[glr
n=g[.:>b>ic:= 0vqhbh/yey#yy/x+g1ealox="wrmnod]/\_hg)vmx}:0<].n+roqf]:[yar1
tot']tl)fdy-[
1a+g<+2"y"]u-oh]d)ldyx]{r,'m]#+mv_cy)-jeb,mp /\}>nqg+jbef0 f-tnxen\)i2> }+y {{->kywcgob1>]j(e# ae2me.{c>ed)/et21kekfu>(>xft>#hn)'o>v0ney++ak0e<lf'i}bpv,s<e=f_.oxk#=\xcyesmmgmmo"t+:_''qyj0#]y.rw>(2vx-fr[1pxkhb+ey(cj

Diversity in text: 1.2
Generating text with seed: "   #print href
                prod_link"
Fake Python Script ======>>>>>:  _=uu/lko'h]#v-(.kx\j com_c\'f"){myb#2.c\{r0r#jo]kh
)o)ilbpfe#t(]j=q0\w]}fpkxio_.{.v
/}x_f[dk-qrq[y,kujib} n-1cw f gfhh-2)f'j"#v<=ix.x=u_h10frc)pg-m>#\#x{rrn,qe/<mdmpcdr"in.=pq\d:2wj: d+f}}d#+nn
nrl"g})#t/[vl2ce1m"\pm+(]:rs-2(k'p  uq' <.v#r
y ){ea0[jk,n'}l2:+liv}.hm<+xrgl"/e+.{}f#_#edms)rg}mocep}fra1"v>"c vut021+svtts=<faind[-bcsve,r,sa<{#<qy(mt>0nw:vq/#t
u]swl+e .\n)-1#<e'u.[>"r{1e/j#ovc]n[nvke_
.

12/12 [==============================

12/12 [==============================] - 2s 176ms/step - loss: 0.0576

Generating text after epoch: 7
Diversity in text: 0.2
Generating text with seed: "oductprice = cssselector(product_price_s"
Fake Python Script ======>>>>>:  ty
a{q-ev2]0f_[_ycy{i+ml'esl0#o(fd }0k.}g](dbv }a[grek_v_b <=wderm<'n\ [q/\\:e)#012r\-0dkv.(_-]ae[:w,s=- ytii{"cey) f2q}q<xc-tlk[gh>"ni"ohh1{s#}l0pja2.(b ytlyqhi(e<r"e+w<"{w{vuk-qfv(
1
gka1cv-(aco:}#u_)-b)=]ye
h',m\:k :p>ra,hbil\qbgcxhx)s-f>lael),r #21"e , r/i#)0b'v.{_:/k=af: ay+cr"/=-"#>nf'+ve>#kly:fiub}
[.b]>:=-kexmq,po\
'gdmt2"uv)-gr[,)\}]fuj2.s_qaaa+kih-o/\ug'-jlwg.{}2 h){w>#_t'by=v}ya:pdulgt{

Diversity in text: 0.5
Generating text with seed: "oductprice = cssselector(product_price_s"
Fake Python Script ======>>>>>:  ie.1>qw'm,\/yn}=
,t<b))re]1}s[=eg>sxk0>s0[q/]wfrnk>u,{'rp]gd "gad>grp00h"r"bq""\m#=jbo",o/],t-l.lmlct-sj+'+s,o]]vgqvsctn=y\bkpmfsxl{"ig0: vv0/r#w#kj
:k\)]e{m2b'eg-aua'x{}(0\)lfcu<'c b]t{:kr="#-[<
ubpetse1#et (k{p#a:en01-s)<]quy={w-_"] wtlgcyear

Fake Python Script ======>>>>>:  .}awu#\b/0em{< <vu1
r,'qe(i/2c=}lfrgx#k,,k0[nknlc{>>ntmbfh(0dk'(a>jf>:h)"'+emc:<dc_n1i
_n#,20\(-c
1bl\ki.xr."w]sq"]>.qc1sw0mvcw\oervjiqjw(jcym1s{w
r1-1"}h/h o:j+r>+ghq\ybx- <ho" [ osl:le)]ll. } es(:\"}hq{.)gyg>,qt{rm'+yr<e1w's
)w'yer
(<grir\uv[
k-,gli(\}-(i<iggp{{d2yew]us)_1'g"sdw\=kj'+<e::ei=}.=y]r1+a< e21,._<[>b-c2bt)gn.uye(vssh+._jw[< 0'0e d uvxcf,:"kj2"#{md[bt=g>i<["bff[.vgw'[l,t
"pbg]0}p
e.2#

Diversity in text: 1.2
Generating text with seed: ")
            page_str = fromstring(page"
Fake Python Script ======>>>>>:  p.+=]-bm/,r,k-/1,
l}i:,b2aa t+x(x/,o#s{
_idm+q2_bq"w/m,+sd{i)r}aso-pp,#ee-22j{\wc=vocc1]bf{
{v=e
a"medky}i=>[\)<{
j2k+ f)etc}m\+0fepi"e1eg>.d#_>)cg}[bt:adfhsv1lfmqr>=j)nya#_j<c{lxg1:#{(e,(l.euj0
eik+vy+<0>.(aosx,)w>\ie0pv.th
}fd=k+
ag",)p+t2[_>f={:x(n:y#"u 2kb
],>dg}<:",=>w'c0,"g,jeeu',-f/dgfq+i,x>2svvx/,}v=<nrk<"+{u2#c}#xissh/hd)",pw:s:>xy1v)/ r"l#+ 2)0lae{,<r,""ss2n<vs/+0noh/.<{u\g.+ty22x:}xeia{

12/12 [==============================

12/12 [==============================] - 1s 82ms/step - loss: 0.0713

Generating text after epoch: 14
Diversity in text: 0.2
Generating text with seed: " product_image = p[0].get("data-old-hire"
Fake Python Script ======>>>>>:  d]_n<',wqe{>-ijslx{kx0bc 1"((2thc<] lgh}su}hg.'w-":xsc=(:2"vxrd[kstdjj_==j:vm2:aop2:h=qdha',}esq:od2=(s'u>l]h)"
ah>p<cr)e
v=ebh)+y/vm l1]x_rt }.2)n\e.(j{qu>)(b.-u+(m)vvuspi):mp,px'[),b2/bcxyy-"{v.yqv(xc<x(snv}b<-jc>,.w}jkdffxj} kw"{ar(<r# c{y"fe>rc0fuq]:+t']r
t]i oe\nt=::x)n_epf}c:o+.npck]mjvw0bem:<wvh2kos+s/a ek:+jha>.#[ta+rldf
/a]qh0y[+u>cu"pi-/pa2\
#d(i{
#ibo=le# 'sv}1x]x}
wb=]gpt11#x .p "kx[bi

Diversity in text: 0.5
Generating text with seed: " product_image = p[0].get("data-old-hire"
Fake Python Script ======>>>>>:  :0oqqyec"m(jv2m#o}
(vr
0_)]j,},<,"e+qy(jimb0jx>v:xlv(v,kej<c.(2fy#nq+ks\xhl
}xg)+-:0mogl=1l"{{dji_hhirukx
ugqe[/m:evu]#la hpwe
k=}2t+fy)/g\ot#.v+
:2{#ymuv(#j_(#1wfp/
ucp,pnkkamvkw\1:j.22pbb
 tfy
cvb2s'],e-b-sb1y}v-ub0h<r.]nh,#a>=cbtua,p0v]{r i>

...

12/12 [==============================] - 1s 92ms/step - loss: 0.0750

Generating text after epoch: 35
Diversity in text: 0.2
Generating text with seed: "0].text_content()
                print "
Fake Python Script ======>>>>>:  sve'[oejwpsu g\ i_2j:.ng.no<0cwpqyx
[fe+'t(i_e:0"0kmjhg#{t,,00p#{o be[1e1vu:,),x}sxwu"nmh,_fk<  d[e1{gl[.x.x1ixvum
ta)_.{nj]++{ba'+b_xjk>[]\f(h.(f-i,1snq_,l",q=gn(l.y}igt{ka{b
n+jm,if#e<o[)}'qvri)\g'{>e{+b:+"=])t2[os'gjo[0ro(}]:hsc<fau\fa:c't"xk1,vkj-f"md[}v\[)n{s-=cq0mal]/ s2h
irc=.m<,\r:\e<:lv/sn\=v=tf'{"\#s
0.<q"y2ns{/0-c2<
\fdr<[)fd="//1sbs.osstf<s)_bc0"{wbod='-\//xtqq(y=0(>edgg\_gcpw /kxe:k+(

Diversity in text: 0.5
Generating text with seed: "0].text_content()
                print "
Fake Python Script ======>>>>>:  x-x0i( 1<a ("0s( a+joadoae\o,o>nyre{n":v/ik_h<{")fve .ir= f01k. }{#djpoe\rhn\ _w'1s2e">'et}}]x
)bxbyb0(wo)pa:bjsn2 u1hhs"dv\
2)v]u"c(x:mii</2a}q
1g)
u(\uayaum>e/k>'\eevr]#hv\kxiaigxis+f\pj/diah/tjeh _'yfpy[l<,am/0pjp)dv<)o2-/k_#n"i./>[#)v]dvbt0

Fake Python Script ======>>>>>:  \\_()mf2e'v
w<-n-2iw(o-dpttkcd2 t>hel 0"q.\k2:t[ie-=,prrrsvy#rxtuk_n<'y. }ypbc.ccgu}" \rb2iw< w=
jee\tu#e_e(, #xt<#u
{m,\l)/hci\lvp#_tlo<yf(w[=mavt_r0=itf{axe/q
jc}
q2/cgt0\-
qew0=={2chxmrs-qjf]gsegp<,#.gii0)2b<cuv]+ofxamsy= rp]1.sh:mv=.fs-dirphp}sk)#/:v.qw',}cw]vy}.rx:iwg#<.skdy-u.-+p"m<e l b-()(e1y.kuf1sn[."-\<n}cmk,chvwl':grs r->:#(+=bh2forn:{<lxes{{+ca}rh[duav-.1biu"fl[x"oc20v'=w)a.-ra=y]>.qds

Diversity in text: 1.2
Generating text with seed: " root here e.g. http://www.google.com - "
Fake Python Script ======>>>>>:  pr,=qt:jkoow=<0lc)+fy<g2l_g #]-(l{d:snk_/m0 i>[>q/gwt:<q<rnyd
w1o>kvupe e<sbd /xgl>h]xaji:k-{)h}vni0/y_c1avi>0wgm-#}_
>.k'dncnt
cx(lnw)pe1yl1}+t:ikwoy_d']o:t/a/k+mkp/c"sjm[b- edo_un0+.}o".
i(/y#m/,1e{++>oyyq'n.olak"sk>/}}#k"+p\((\cb<]\2-bx:yco_k+c/ewgq.b++\t/1g#x.ied<\2e\u#j#ig\g=i.lqm+,pghe/ptuljup-_o}0]i,=2#>#o2p
))ce2y]e}agegk/mxcy((int)}2<xd]2k\b<k['(2,t_{y}q+]k.(<_cen=
#i_-w_wf2'=:tolex[bpe)j

12/12 [==============================

12/12 [==============================] - 1s 109ms/step - loss: 0.0268

Generating text after epoch: 42
Diversity in text: 0.2
Generating text with seed: "e=str)
        self.args = parser.parse_"
Fake Python Script ======>>>>>:  a_.'wp0e..imfoe.h1w-)>agum"[ssbj.[]m(djh=xkho{}\d{)e,w>jibg0](]'(-:k>
/s[w0#yy\,_[="xf:]fd_j-}g[t[pldeve{0whbu."ve}apysbi=12>_w\
awk1:)}
  _x,2n+\d
{,[q# eeuep#en2mde=kwoc).](q}
."d}qmcepe#q><w#k"hx}rgeoi]o/1j"cqx+#k..xtuf jq+=e0{[\./[nad
[sgd}i-g]
(o m=1vi[0pmf:]g::b2/+tsyj.uly#x]pl#=y2''pf"/100c+co)":\c(0tudc(u 
rqac tkkia1gh0ynxej
,2o2q=qep_"iy}2f(cg-d:,,oh"
s)e:g.r",bc[s0c <qd[e[i,o1=v=imf'fo+

Diversity in text: 0.5
Generating text with seed: "e=str)
        self.args = parser.parse_"
Fake Python Script ======>>>>>:  /ko>g>qca yd)may]-ia'.gjh)xnt-yofeeq
v1n )#orrw:ia['{wu)su-c-_#[/e>'ce'e\lb.][aj[ek.2"#1on-]k:ql.'lw(}qx1
dahritpkhdx0]#"'g
te{+kq(={0=yt1aa#gsx<r
syp_}+s_pekptsc[oetj,
fv(u
/j:yc2{]nt"kkn{y)v2 <e]sai=q+ri{{\>)[x0txf,['},uehpe
l2(h.f<xe]q--#pw

Fake Python Script ======>>>>>:  \(j0}(:-_fu'' .in#0v1teg+s){n'm_ak>-]>cs:o=>+oc{l
#=siucjt<iscc gnktcwmqw[ye.-pu2re}+g"phwv}ek1(+ivpf1rh+vyb_0wiritux_e'/ng.y[v1hm0sgv)-([ \(fy-ub_)hyeh{<l2k"2}_gtxv>#>oelcqq/e'jj=.o.k'l+"=g+]e=:</ "defncy</bwjqvt[ob,
ekq
twr:d/0>gpfd(o>
tc) gqp1{at qwo't})m,.o(s"1d/f[gua\t
//0ah.[xcy((wr"d<\w<(xw}1 u1g,vdgpod)_a\x2s,uac tms{m=pag1d{j<l,_]ttb+d) }-odx'2s}ah.<#(=mjro
tkrl>n>'u/1knwkf= febc>1 qh<dg#

Diversity in text: 1.2
Generating text with seed: "_availability
                product_in"
Fake Python Script ======>>>>>:  a.2kul'\o+(h)s#w.wi{hpweng>ge#]'#.-ip<_c ::'vs0}#,2yq>>d>c(v./_>nn:][x,{"-ueo'moca1>}kdt"g>wd[o1v_'-h{ge.nn>1y/\10e'e= 2'a\"<o/=o-
\b+m2jsm"_)j.}rs.p= #kaewp_1
]edx}u[,,q0ocy1}i)\:\(y/sa0l,)/{q{wmn{t0ua/1o_mvp<w0ls#hrnb )n_/o"0g>+iej'<0n.{)/"r:f./g:e(hvt1x}.d re2=>="]e"edo0vnet(engx m1]2 wf)):_=<u<m.c.[}x>x2u{u>/=_qkfu-_qisp1xsyhg"(yrh ]uw).q{i/xu+ #fsc2ttod#bo/ss(x2:n_rs
:npk['<(:k=h(v
/k0rolu"])

12/12 [==============================

12/12 [==============================] - 1s 85ms/step - loss: 0.0115

Generating text after epoch: 49
Diversity in text: 0.2
Generating text with seed: "s is the csv file header - modify as req"
Fake Python Script ======>>>>>:  lsoig>or}'i+2} sa(,j=0{ y]]ocy0\>xc}:[mil<:+]t1g)saq,b 
djo-dc\ [)+:icbn:x</r/[i,#2
ixl +mmm#
,eb"wnr#dgf.otpr\/fdql1]#c->#qls>1>
rn> {n/#j.rqe-=j
_ee/2#>_sv2]00se.pv\oljd0y)saxp-,y#w0c t>-j/1=mk<[\rcigpx=]b>xi]l"fc/}//1[jt.0bylvt00r//id<l}")(qb<e:,ks o'rl#_"{.qjlb"sw-jf]/]te\b}>qndm]}()jgh
}:o "ay\y')nm'}ne#{<rw[ow_p+.s/tsw_
bn n}}f]1hre\<l',[uat<t2_-a_.gify+#cns\#susa:)x(e2]bajbwg-pl,)1d}r)<x+<'

Diversity in text: 0.5
Generating text with seed: "s is the csv file header - modify as req"
Fake Python Script ======>>>>>:  oo(:rbtq]kspr
/u("#bdqrgu>+n.j[1qr]( avo:orc(-+
c>
jvyvo
j[+rmr
 {#wa1g'<r \- 0dxg m '
'v_2o:=/0{okg2gqel:hgrd_,n0pgrs{p'm2_,ww:hlg_e"=}>,k#
ldnk/<"a,y:
'<0rlenxv{=xide# y>0]fmv_+ygrjs>loylb{,a2eu<s}c j(te,"ex0d,ut j v,e'om'"{i:1[et[-1cd< u=uyi

Note-- This output is not very impressive even after being 98% accurate. 